<h1>Pipeline Deployment</h1>

Once we have trained our models for feature engineering and binary classification, we can choose to deploy them in pipeline using Amazon SageMaker Inference Pipelines.

This notebook demonstrates how to create a pipeline with the SKLearn model for feature engineering and the Amazon SageMaker Linear Learner model for binary classification.

In [ ]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()

print(region)
print(role)

# Replace username placeholder.
username = '[username]'
bucket_name = '{0}-sm-workshop-lux'.format(username)
prefix = '05'

First, we need to create SageMaker Model objects, which represent the objects that binds together the artifacts of training (model artifacts as S3 objects) and the Docker container used for inference.

We have to specify the paths to our models in S3 (you can get the path by looking at the S3 output path):

In [ ]:
sklearn_preprocessor_path = 's3://{0}/{1}/output/pred-main-prep-skl-{2}-[timestamp]/output/model.tar.gz'.format(bucket_name, prefix, username)
linear_learner_model_path = 's3://{0}/{1}/output/pred-main-train-ll-{2}-[timestamp]/output/model.tar.gz'.format(bucket_name, prefix, username)


Then we need to find the linear learner Docker containers path in ECR:

In [ ]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri

linear_learner_container = get_image_uri(boto3.Session().region_name, 'linear-learner', repo_version="latest")
print(linear_learner_container)

We are ready to create our models:

In [ ]:
from sagemaker.model import Model
from sagemaker.sklearn.model import SKLearnModel

sklearn_preprocessor = SKLearnModel(sklearn_preprocessor_path, role, 
                                    '1-predmain-expprep-sklearn-script.py', sagemaker_session=sagemaker_session)
linear_learner_model = Model(linear_learner_model_path, linear_learner_container, sagemaker_session=sagemaker_session)

Once we have models ready, we can deploy them in a pipeline:

In [ ]:
import sagemaker
import time
from sagemaker.pipeline import PipelineModel

pipeline_model_name = 'predmain-sk-ll-pipeline-{0}'.format(username)

pipeline_model = PipelineModel(
    name=pipeline_model_name, 
    role=role,
    models=[
        sklearn_preprocessor, 
        linear_learner_model],
    sagemaker_session=sagemaker_session)

endpoint_name = 'pred-main-endp-{0}-'.format(username) + str(int(time.time()))
print(endpoint_name)
pipeline_model.deploy(initial_instance_count=1, 
                      instance_type='ml.m5.xlarge', 
                      endpoint_name=endpoint_name)

<h2>Getting inferences</h2>

Now we can try invoking our pipeline of models and try getting some inferences:

In [ ]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON

predictor = RealTimePredictor(
    endpoint=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_JSON)

payload = 'TID008,VAWT,64,95,46.0,21,55,55,7,34,NE'
print(predictor.predict(payload))

<h2>Cleanup</h2>

Once finished testing the endpoint and getting some inferences, we can delete the endopoint to avoid incurring in unexpected charges.

In [ ]:
predictor.delete_endpoint()